### Матричные факторизации

В данной работе вам предстоит познакомиться с практической стороной матричных разложений.
Работа поделена на 4 задания:
1. Вам необходимо реализовать SVD разложения используя SGD на explicit данных
2. Вам необходимо реализовать матричное разложения используя ALS на implicit данных
3. Вам необходимо реализовать матричное разложения используя BPR(pair-wise loss) на implicit данных
4. Вам необходимо реализовать матричное разложения используя WARP(list-wise loss) на implicit данных

#### Пример со встроенными функциями

In [69]:
import implicit
import pandas as pd
import numpy as np
import scipy.sparse as sp

from tqdm import tqdm


В данной работе мы будем работать с explicit датасетом movieLens, в котором представленны пары user_id movie_id и rating выставленный пользователем фильму

Скачать датасет можно по ссылке https://grouplens.org/datasets/movielens/1m/

In [70]:
ratings = pd.read_csv('data/ratings.dat', delimiter='::', header=None,
        names=['user_id', 'movie_id', 'rating', 'timestamp'],
        usecols=['user_id', 'movie_id', 'rating'], engine='python')

In [71]:
movie_info = pd.read_csv('data/movies.dat', delimiter='::', header=None,
        names=['movie_id', 'name', 'category'], engine='python')

movie_info

,movie_id,name,category
0,1,Toy Story (1995),Animation|Children's|Comedy
1,2,Jumanji (1995),Adventure|Children's|Fantasy
2,3,Grumpier Old Men (1995),Comedy|Romance
3,4,Waiting to Exhale (1995),Comedy|Drama
4,5,Father of the Bride Part II (1995),Comedy
...,...,...,...
3878,3948,Meet the Parents (2000),Comedy
3879,3949,Requiem for a Dream (2000),Drama
3880,3950,Tigerland (2000),Drama
3881,3951,Two Family House (2000),Drama


Explicit данные

In [72]:
ratings

,user_id,movie_id,rating
0,1,1193,5
1,1,661,3
2,1,914,3
3,1,3408,4
4,1,2355,5
...,...,...,...
1000204,6040,1091,1
1000205,6040,1094,5
1000206,6040,562,5
1000207,6040,1096,4


Для того, чтобы преобразовать текущий датасет в Implicit, давайте считать что позитивная оценка это оценка >=4

In [73]:
implicit_ratings = ratings.loc[(ratings['rating'] >= 4)]

In [74]:
implicit_ratings.head(10)

,user_id,movie_id,rating
0,1,1193,5
3,1,3408,4
4,1,2355,5
6,1,1287,5
7,1,2804,5
8,1,594,4
9,1,919,4
10,1,595,5
11,1,938,4
12,1,2398,4


Удобнее работать с sparse матричками, давайте преобразуем DataFrame в CSR матрицы

In [75]:
users = implicit_ratings["user_id"]
movies = implicit_ratings["movie_id"]
user_item = sp.coo_matrix((np.ones_like(users), (users, movies)))
user_item_t_csr = user_item.T.tocsr()
user_item_csr = user_item.tocsr()

user_item_csr

<6041x3953 sparse matrix of type '<class 'numpy.int64'>'
	with 575281 stored elements in Compressed Sparse Row format>

В качестве примера воспользуемся ALS разложением из библиотеки implicit

Зададим размерность латентного пространства равным 64, это же определяет размер user/item эмбедингов

In [76]:
model = implicit.als.AlternatingLeastSquares(factors=64, iterations=100, calculate_training_loss=True)

В качестве loss здесь всеми любимый RMSE

In [77]:
model.fit(user_item_t_csr)

Построим похожие фильмы по 1 movie_id = Истории игрушек

In [78]:
movie_info.head(5)

,movie_id,name,category
0,1,Toy Story (1995),Animation|Children's|Comedy
1,2,Jumanji (1995),Adventure|Children's|Fantasy
2,3,Grumpier Old Men (1995),Comedy|Romance
3,4,Waiting to Exhale (1995),Comedy|Drama
4,5,Father of the Bride Part II (1995),Comedy


In [79]:
get_similars = lambda item_id, model : [movie_info[movie_info["movie_id"] == x[0]]["name"].to_string()
                                        for x in model.similar_items(item_id)]

Как мы видим, симилары действительно оказались симиларами.

Качество симиларов часто является хорошим способом проверить качество алгоритмов.

P.S. Если хочется поглубже разобраться в том как разные алгоритмы формируют разные латентные пространства, рекомендую загружать полученные вектора в tensorBoard и смотреть на сформированное пространство

In [80]:
get_similars(1, model)

['0    Toy Story (1995)',
 '3045    Toy Story 2 (1999)',
 "2286    Bug's Life, A (1998)",
 '33    Babe (1995)',
 '584    Aladdin (1992)',
 '2315    Babe: Pig in the City (1998)',
 '360    Lion King, The (1994)',
 '1526    Hercules (1997)',
 '2252    Pleasantville (1998)',
 '2692    Iron Giant, The (1999)']

Давайте теперь построим рекомендации для юзеров

Как мы видим юзеру нравится фантастика, значит и в рекомендациях ожидаем увидеть фантастику

In [81]:
get_user_history = lambda user_id, cur_ratings : [movie_info[movie_info["movie_id"] == x]["name"].to_string()
                                            for x in cur_ratings[cur_ratings["user_id"] == user_id]["movie_id"]]

In [82]:
get_user_history(4, implicit_ratings)

['3399    Hustler, The (1961)',
 '2882    Fistful of Dollars, A (1964)',
 '1196    Alien (1979)',
 '1023    Die Hard (1988)',
 '257    Star Wars: Episode IV - A New Hope (1977)',
 '1959    Saving Private Ryan (1998)',
 '476    Jurassic Park (1993)',
 '1180    Raiders of the Lost Ark (1981)',
 '1885    Rocky (1976)',
 '1081    E.T. the Extra-Terrestrial (1982)',
 '3349    Thelma & Louise (1991)',
 '3633    Mad Max (1979)',
 '2297    King Kong (1933)',
 '1366    Jaws (1975)',
 '1183    Good, The Bad and The Ugly, The (1966)',
 '2623    Run Lola Run (Lola rennt) (1998)',
 '2878    Goldfinger (1964)',
 '1220    Terminator, The (1984)']

Получилось!

Мы действительно порекомендовали пользователю фантастику и боевики, более того встречаются продолжения тех фильмов, которые он высоко оценил

In [83]:
get_recommendations = lambda user_id, model : [movie_info[movie_info["movie_id"] == x[0]]["name"].to_string()
                                               for x in model.recommend(user_id, user_item_csr)]

In [84]:
get_recommendations(4, model)

['585    Terminator 2: Judgment Day (1991)',
 '1271    Indiana Jones and the Last Crusade (1989)',
 '1284    Butch Cassidy and the Sundance Kid (1969)',
 '1182    Aliens (1986)',
 '1178    Star Wars: Episode V - The Empire Strikes Back...',
 '2502    Matrix, The (1999)',
 '1884    French Connection, The (1971)',
 '1179    Princess Bride, The (1987)',
 '3458    Predator (1987)',
 '847    Godfather, The (1972)']

Теперь ваша очередь реализовать самые популярные алгоритмы матричных разложений

Что будет оцениваться:
1. Корректность алгоритма
2. Качество получившихся симиларов
3. Качество итоговых рекомендаций для юзера

### Задание 1. Не используя готовые решения, реализовать SVD разложение используя SGD на explicit данных

#### Data preprocessing

Рассмотрим наши данные поближе, чтобы обнаружить вдруг где-то в них есть ошибка

Начнем с поля 'rating' оно принимает значение от 1-5, неточностей нет

In [85]:
ratings.rating.describe()

count    1.000209e+06
mean     3.581564e+00
std      1.117102e+00
min      1.000000e+00
25%      3.000000e+00
50%      4.000000e+00
75%      4.000000e+00
max      5.000000e+00
Name: rating, dtype: float64

Теперь рассмотрим все id и начнем с 'user_id'

'user_id' оказались корректными, без пропусков от 1 до последнего юзера

In [86]:
users = np.unique(ratings["user_id"])
len(set(range(np.min(users), np.max(users) + 1)) - set(users))

0

Аналогично проделаем с 'movie_id' и увидим что есть пропущенные id

In [87]:
movies = np.unique(ratings["movie_id"])
len(set(range(np.min(movies), np.max(movies) + 1)) - set(movies))

246

Сделаем новые корректные id для фильмов, для этого заведем словари

In [88]:
user_to_id = dict(zip(users, range(len(users))))
id_to_user = dict(zip(range(len(users)), users))

movie_to_id = dict(zip(movies, range(len(movies))))
id_to_movie = dict(zip(range(len(movies)), movies))

Изменим старые id на новые в movie_info

In [89]:
movie_info['movie_id'] = movie_info['movie_id'].map(movie_to_id)

Составим матрицу рейтингов для новых id

In [90]:
user_item = sp.coo_matrix((ratings["rating"], ([user_to_id[user] for user in ratings["user_id"]], [movie_to_id[movie] for movie in ratings["movie_id"]])))
user_item_explicit_t_csr = user_item.T.tocsr()
user_item_explicit_csr = user_item.tocsr()
user_item_explicit_csr

<6040x3706 sparse matrix of type '<class 'numpy.int64'>'
	with 1000209 stored elements in Compressed Sparse Row format>

Создадим класс Recommender и отнаследуем от него все реализации, чтобы вынести общий код

In [91]:
class Recommender:
    def __init__(self, features_dim, lambda_reg):
        self.features_dim = features_dim
        self.lambda_reg = lambda_reg
        self.width, self.height, self.size = None, None, None
        self.W, self.H = None, None
        self.nonzero, self.data = None, None

    def prepare_fit(self, X_csr):
        self.width, self.height, self.size = X_csr.shape[0], X_csr.shape[1], len(X_csr.data)
        self.W = np.random.uniform(0.0, 1.0 / np.sqrt(self.features_dim), (self.width, self.features_dim))
        self.H = np.random.uniform(0.0, 1.0 / np.sqrt(self.features_dim), (self.height, self.features_dim))
        self.nonzero = X_csr.nonzero()
        self.data = X_csr.toarray()

    def similar_items(self, item_id, num = 10):
        return np.asarray([np.linalg.norm(col - self.H[item_id]) for col in self.H]).argsort()[:num]

    def recommend(self, user_id, cur_user_item_csr, num = 10):
        items_ids = cur_user_item_csr[user_id].indices
        bias_W = self.bias_W[user_id] if hasattr(self, 'bias_W') else 0
        bias_H = self.bias_H if hasattr(self, 'bias_H') else 0
        mu = self.mu if hasattr(self, 'mu') else 0
        candidates = self.W[user_id] @ self.H.T + bias_W + bias_H + mu
        return list(filter(lambda candidate: candidate not in items_ids , candidates.argsort()[::-1]))[:num]

In [92]:
class SVD(Recommender):
    def __init__(self, features_dim = 64, eps = 0.01, learning_rate = 0.01, lambda_reg = 0.01, batches = 20, batch_size = 500000, seed = 1):
        super().__init__(features_dim, lambda_reg)
        self.eps = eps
        self.learning_rate = learning_rate
        self.batches = batches
        self.batch_size = batch_size
        self.bias_W, self.bias_H, self.mu = None, None, None
        np.random.seed(seed)

    def fit(self, X_csr):
        super().prepare_fit(X_csr)
        bias_W, bias_H = np.zeros(self.width), np.zeros(self.height)
        W, H = self.W, self.H

        mu = X_csr.data.mean()
        indices = [i for i in range(self.size)]
        pbar = tqdm(range(self.batches))
        for _ in pbar:
            np.random.shuffle(indices)
            sum_error = 0
            for index in indices[:self.batch_size]:
                value, w_id, h_id = X_csr.data[index], self.nonzero[0][index], self.nonzero[1][index]
                error = W[w_id] @ H[h_id] + bias_W[w_id] + bias_H[h_id] + mu - value
                sum_error += error ** 2

                W[w_id] = W[w_id] - self.learning_rate * (error * H[h_id] + self.lambda_reg * W[w_id])
                H[h_id] = H[h_id] - self.learning_rate * (error * W[w_id] + self.lambda_reg * H[h_id])

                bias_W[w_id] = bias_W[w_id] - self.learning_rate * (error + self.lambda_reg * bias_W[w_id])
                bias_H[h_id] = bias_H[h_id] - self.learning_rate * (error + self.lambda_reg * bias_H[h_id])
                mu -= self.learning_rate * error

            rmse = np.sqrt(sum_error / self.batch_size)
            pbar.set_description("RMSE: %f" % rmse)
            if rmse < self.eps:
                break
        self.W, self.H, self.bias_W, self.bias_H, self.mu = W, H, bias_W, bias_H, mu

In [93]:
svd = SVD()
svd.fit(user_item_explicit_csr)

RMSE: 0.730044: 100%|██████████| 20/20 [03:02<00:00,  9.14s/it]


В качестве тестирования похожих фильмов будем рассматривать фильм из примера у которого был id = 1

In [94]:
test_movie_id = movie_to_id[1]
movie_info[movie_info['movie_id'] == test_movie_id]

,movie_id,name,category
0,0.0,Toy Story (1995),Animation|Children's|Comedy


In [95]:
similiars = svd.similar_items(test_movie_id)
movie_info.set_index('movie_id').loc[similiars, :]

,name,category
movie_id,,
0.0,Toy Story (1995),Animation|Children's|Comedy
2898.0,Toy Story 2 (1999),Animation|Children's|Comedy
2162.0,"Bug's Life, A (1998)",Animation|Children's|Comedy
1439.0,Hercules (1997),Adventure|Animation|Children's|Comedy|Musical
3061.0,"Tigger Movie, The (2000)",Animation|Children's
1909.0,"Rescuers, The (1977)",Animation|Children's
1876.0,"Incredible Journey, The (1963)",Adventure|Children's
1727.0,Mulan (1998),Animation|Children's
959.0,"Three Caballeros, The (1945)",Animation|Children's|Musical


В качестве тестирования рекомендация для пользователя будем рассматривать пользователя из примера у которого был id = 4

In [96]:
test_user_id = user_to_id[4]

In [97]:
recommendations = svd.recommend(test_user_id, user_item_csr)
movie_info.set_index('movie_id').loc[recommendations, :]

,name,category
movie_id,,
1117.0,To Kill a Mockingbird (1962),Drama
2698.0,Sanjuro (1962),Action|Adventure
802.0,"Godfather, The (1972)",Action|Crime|Drama
1839.0,Seven Samurai (The Magnificent Seven) (Shichin...,Action|Drama
861.0,Sunset Blvd. (a.k.a. Sunset Boulevard) (1950),Film-Noir
1113.0,12 Angry Men (1957),Drama
843.0,Rear Window (1954),Mystery|Thriller
309.0,"Shawshank Redemption, The (1994)",Drama
1848.0,Saving Private Ryan (1998),Action|Drama|War


### Задание 2. Не использую готовые решения, реализовать матричное разложение используя ALS на implicit данных

#### Data preprocessing

Подготовим implicit данные также, как мы подготовили explicit данные в предыдущем пункте

In [98]:
users = [user_to_id[user] for user in implicit_ratings['user_id']]
movies = [movie_to_id[movie] for movie in implicit_ratings['movie_id']]
user_item = sp.coo_matrix((np.ones_like(users), (users, movies)))
user_item_t_csr = user_item.T.tocsr()
user_item_csr = user_item.tocsr()
user_item_csr

<6040x3706 sparse matrix of type '<class 'numpy.int64'>'
	with 575281 stored elements in Compressed Sparse Row format>

In [99]:
class ALS(Recommender):
    def __init__(self, features_dim = 64, lambda_reg = 0.01, iterations = 5, alpha = 10):
        super().__init__(features_dim, lambda_reg)
        self.iterations = iterations
        self.alpha = alpha

    def fit(self, X_csr):
        super().prepare_fit(X_csr)
        pbar = tqdm(range(self.iterations))
        reg_matrix = self.lambda_reg * np.eye(self.features_dim)
        for _ in pbar:
            Ht = self.H.T
            Ht_H = Ht @ self.H
            for w_id in range(self.width):
                p_u = self.data[w_id, :]
                C_u = sp.diags(1 + self.alpha * p_u)
                modification = Ht_H + Ht @ (C_u - sp.identity(self.height)) @ self.H
                left_inv_part = np.linalg.inv(modification + reg_matrix)
                self.W[w_id] = left_inv_part @ Ht @ C_u @ p_u

            Wt = self.W.T
            Wt_W = self.W.T @ self.W
            for h_id in range(self.height):
                p_i = self.data[:, h_id]
                C_i = sp.diags(1 + self.alpha * p_i)
                modification = Wt_W + Wt @ (C_i - sp.identity(self.width)) @ self.W
                left_inv_part = np.linalg.inv(modification + reg_matrix)
                self.H[h_id] = left_inv_part @ Wt @ C_i @ p_i

            predictions = self.W @ self.H.T
            rmse = np.sqrt(np.mean((predictions[self.nonzero] - X_csr.data) ** 2))
            pbar.set_description("RMSE: %f" % rmse)

In [100]:
als = ALS()
als.fit(user_item_csr)

RMSE: 0.303626: 100%|██████████| 5/5 [03:05<00:00, 37.11s/it]


In [101]:
similiars = als.similar_items(test_movie_id)
movie_info.set_index('movie_id').loc[similiars, :]

,name,category
movie_id,,
0.0,Toy Story (1995),Animation|Children's|Comedy
2898.0,Toy Story 2 (1999),Animation|Children's|Comedy
2162.0,"Bug's Life, A (1998)",Animation|Children's|Comedy
574.0,Aladdin (1992),Animation|Children's|Comedy|Musical
1173.0,Groundhog Day (1993),Comedy|Romance
581.0,Beauty and the Beast (1991),Animation|Children's|Musical
2128.0,Pleasantville (1998),Comedy
33.0,Babe (1995),Children's|Comedy|Drama
354.0,"Lion King, The (1994)",Animation|Children's|Musical


In [102]:
recommendations = als.recommend(test_user_id, user_item_csr)
movie_info.set_index('movie_id').loc[recommendations, :]

,name,category
movie_id,,
1106.0,Star Wars: Episode V - The Empire Strikes Back...,Action|Adventure|Drama|Sci-Fi|War
575.0,Terminator 2: Judgment Day (1991),Action|Sci-Fi|Thriller
2374.0,"Matrix, The (1999)",Action|Sci-Fi|Thriller
1110.0,Aliens (1986),Action|Sci-Fi|Thriller|War
1120.0,Star Wars: Episode VI - Return of the Jedi (1983),Action|Adventure|Romance|Sci-Fi|War
443.0,"Fugitive, The (1993)",Action|Thriller
1199.0,Indiana Jones and the Last Crusade (1989),Action|Adventure
106.0,Braveheart (1995),Action|Drama|War
2334.0,Planet of the Apes (1968),Action|Sci-Fi


### Задание 3. Не использую готовые решения, реализовать матричное разложение BPR на implicit данных

In [103]:
from sklearn.metrics import roc_auc_score


class BPR(Recommender):
    def __init__(self, features_dim = 64, learning_rate = 0.03, lambda_reg = 0.00001, iterations = 10, seed = 42):
        super().__init__(features_dim, lambda_reg)
        self.learning_rate = learning_rate
        self.iterations = iterations
        self.h_negatives = None
        np.random.seed(seed)

    def fit(self, X_csr):
        super().prepare_fit(X_csr)
        Ds = [(w_id, h_id) for w_id, h_id in zip(self.nonzero[0], self.nonzero[1])]

        indices = [i for i in range(self.height)]
        self.h_negatives = dict([(w_id,np.array(list(set(indices) - set(X_csr[w_id, :].indices)))) for w_id in range(self.width)])

        pbar = tqdm(range(self.iterations))
        for _ in pbar:
            np.random.shuffle(Ds)
            for w_id, i in Ds:
                j, coef = self.get_second_index(w_id, i)

                self.W[w_id] += self.learning_rate * (coef * (self.H[i] - self.H[j]) + self.lambda_reg * self.W[w_id])
                self.H[i] += self.learning_rate * (coef * self.W[w_id] + self.lambda_reg * self.H[i])
                self.H[j] += self.learning_rate * (coef * -self.W[w_id] + self.lambda_reg * self.H[j])

            pbar.set_description("AUC: %f" % self.auc_score(X_csr))

    def get_second_index(self, w_id, i):
        j = np.random.choice(self.h_negatives[w_id])
        cur_r = (self.W[w_id] @ self.H[i]) - (self.W[w_id] @ self.H[j])
        coef = np.exp(-cur_r) / (1 + np.exp(-cur_r))
        return j, coef

    def auc_score(self, X_csr):
        full_auc = 0.0
        for w_id in range(self.width):
            y_pred = self.W[w_id] @ self.H.T
            y_true = np.zeros(self.height)
            y_true[X_csr[w_id, :].indices] = 1
            try:
                full_auc += roc_auc_score(y_true, y_pred)
            except ValueError: # if only one class present in y_true
                pass
        return full_auc / self.width

In [104]:
bpr = BPR()
bpr.fit(user_item_csr)

AUC: 0.928616: 100%|██████████| 10/10 [05:06<00:00, 30.61s/it]


In [105]:
similiars = bpr.similar_items(test_movie_id)
movie_info.set_index('movie_id').loc[similiars, :]

,name,category
movie_id,,
0.0,Toy Story (1995),Animation|Children's|Comedy
1107.0,"Princess Bride, The (1987)",Action|Adventure|Comedy|Romance
2162.0,"Bug's Life, A (1998)",Animation|Children's|Comedy
2898.0,Toy Story 2 (1999),Animation|Children's|Comedy
1025.0,E.T. the Extra-Terrestrial (1982),Children's|Drama|Fantasy|Sci-Fi
33.0,Babe (1995),Children's|Comedy|Drama
2775.0,Who Framed Roger Rabbit? (1988),Adventure|Animation|Film-Noir
574.0,Aladdin (1992),Animation|Children's|Comedy|Musical
1178.0,Back to the Future (1985),Comedy|Sci-Fi


In [106]:
recommendations = bpr.recommend(test_user_id, user_item_csr)
movie_info.set_index('movie_id').loc[recommendations, :]

,name,category
movie_id,,
1106.0,Star Wars: Episode V - The Empire Strikes Back...,Action|Adventure|Drama|Sci-Fi|War
2651.0,American Beauty (1999),Comedy|Drama
2374.0,"Matrix, The (1999)",Action|Sci-Fi|Thriller
575.0,Terminator 2: Judgment Day (1991),Action|Sci-Fi|Thriller
579.0,"Silence of the Lambs, The (1991)",Drama|Thriller
1485.0,L.A. Confidential (1997),Crime|Film-Noir|Mystery|Thriller
802.0,"Godfather, The (1972)",Action|Crime|Drama
309.0,"Shawshank Redemption, The (1994)",Drama
1107.0,"Princess Bride, The (1987)",Action|Adventure|Comedy|Romance


### Задание 4. Не использую готовые решения, реализовать матричное разложение WARP на implicit данных

In [107]:
class WARP(BPR):
    def __init__(self, features_dim = 64, learning_rate = 0.03, lambda_reg = 0.00001, iterations = 10, seed = 42, eps = 1, samples = 10):
        super().__init__(features_dim, learning_rate, lambda_reg, iterations, seed)
        self.eps = eps
        self.samples = samples

    def get_second_index(self, w_id, i):
        score_i = self.W[w_id] @ self.H[i]
        j, score_j = None, None
        for n in range(1, self.samples):
            j = np.random.choice(self.h_negatives[w_id])
            score_j = self.W[w_id] @ self.H[j]
            if score_j - self.eps > score_i:
                cur_r = score_i - score_j
                coef = np.exp(-cur_r) / (1 + np.exp(-cur_r))
                return j, coef * np.log(self.samples / n)
        cur_r = score_i - score_j
        return j, np.exp(-cur_r) / (1 + np.exp(-cur_r))

In [108]:
warp = WARP()
warp.fit(user_item_csr)

AUC: 0.953620: 100%|██████████| 10/10 [12:30<00:00, 75.08s/it]


In [109]:
similiars = warp.similar_items(test_movie_id)
movie_info.set_index('movie_id').loc[similiars, :]

,name,category
movie_id,,
0.0,Toy Story (1995),Animation|Children's|Comedy
2162.0,"Bug's Life, A (1998)",Animation|Children's|Comedy
574.0,Aladdin (1992),Animation|Children's|Comedy|Musical
2556.0,"Iron Giant, The (1999)",Animation|Children's
2483.0,Tarzan (1999),Animation|Children's
2898.0,Toy Story 2 (1999),Animation|Children's|Comedy
2102.0,Antz (1998),Animation|Children's
639.0,James and the Giant Peach (1996),Animation|Children's|Musical
581.0,Beauty and the Beast (1991),Animation|Children's|Musical


In [110]:
recommendations = warp.recommend(test_user_id, user_item_csr)
movie_info.set_index('movie_id').loc[recommendations, :]

,name,category
movie_id,,
1485.0,L.A. Confidential (1997),Crime|Film-Noir|Mystery|Thriller
1106.0,Star Wars: Episode V - The Empire Strikes Back...,Action|Adventure|Drama|Sci-Fi|War
1120.0,Star Wars: Episode VI - Return of the Jedi (1983),Action|Adventure|Romance|Sci-Fi|War
575.0,Terminator 2: Judgment Day (1991),Action|Sci-Fi|Thriller
1110.0,Aliens (1986),Action|Sci-Fi|Thriller|War
593.0,Fargo (1996),Crime|Drama|Thriller
2651.0,American Beauty (1999),Comedy|Drama
106.0,Braveheart (1995),Action|Drama|War
287.0,Pulp Fiction (1994),Crime|Drama
